# Whereabouts Plans Template
This notebook will demonstrate how to create a whereabouts plan automatically.

<div style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg/640px-Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg" /></div>

## Introduction
The purpose of this notebook is to create a Street and Bridge Work Order plans based on segment IDs and additional comments on long line. Markings feature layers are published in the City of Austin ArcGIS Portal page available for public view as well. 

The data should already be available in the folder path as an excel spreadsheet. If the spreadsheet exists, a map document will be configured for spreadsheet use.

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [pathlib](https://docs.python.org/3/library/pathlib.html) to find path to excel document if it exists
- [archook](https://github.com/JamesRamm/archook) to search for arcgis and makes arcpy available to python
- [arcpy](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm) to create whereabouts markings plans using ESRI ArcMap Desktop software
- [openpyxl](https://openpyxl.readthedocs.io/en/stable/) to open excel files of markings and SBO data

In [1]:
import pandas as pd
from pathlib import Path
import datetime
import math
from functools import reduce

import archook 
archook.get_arcpy()
import arcpy

from PyPDF2 import PdfFileMerger
import openpyxl
from PIL import Image 
import os 

## Constants

The date by month and day constant will determine the file pdf name to use as a dataframe. Folder path will determine where the plans will be created depending on the year. This is set to the top for the purpose of changing these constants as needed.

<i>The table below explains the purpose of each constant.</i>

| Constant | Description   |
|:--------:|----|
| <b>MONTH, DAY, YEAR</b> |Date used to find PDF in month-day format and file path based on year|
|<b>FOLDER</b>      |File directory used to import SBO whereabouts reports from email|
|<b>FILE_NAME</b>   |File directory name used to extact SBO whereabouts reports from file|
|<b>MXD</b>   |Map document used to create whereabouts plans template|

In [2]:
%store -r MONTH
%store -r DAY
%store -r YEAR
%store -r FOLDER
%store -r FILE_NAME
MXD = r"C:\Users\Govs\Projects\Whereabouts\Whereabouts_Cover_Template.mxd"
PLAN_PATH = r"G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\{}".format(YEAR)

## Methods
These functions will be used to extract and transform the data into a feasible format.

<i>The table below explains the purpose of each:</i>

| Method | Description   |
|:--------:|----|
|<b>df_text</b> |Changes element text to the pandas dataframe|
|<b>cover_aerial</b> |Zooms to selected features and extent for the dataframe|

In [3]:
# Formats element text to match pandas dataframe
def df_text(r):
    date = datetime.datetime.now()
    for e in arcpy.mapping.ListLayoutElements(mxd,'TEXT_ELEMENT'):
            if e.name == 'SPECIFICATIONS':
                e.text = str(r["SPECIFICATIONS"])
            elif e.name == 'CREATED DATE':
                e.text =  "{}/{}/{}".format(date.month,date.day,date.year)
            elif e.name == 'REQUESTOR ID':
                e.text = str(r["Location ID"])
            elif e.name == 'LOCATION':
                e.text = "{} from {} to {}".format(r["Street"],r["From"],r["To"])
            elif e.name == 'author':
                e.text = 'Susanne Gov'
            elif e.name == 'WORK GROUPS':
                e.text = str(r['WORK GROUPS'])

# Refreshes dataframe map aerial                
def cover_aerial(mapdoc):
    dataframe = arcpy.mapping.ListDataFrames(mapdoc,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    arcpy.RefreshActiveView()

## Import Datasets

In [4]:
if Path(FILE_NAME + '.xlsx').exists():
    df = pd.read_excel(FILE_NAME + '.xlsx','Cover', index_col=0).reset_index()
    df1 = pd.read_excel(FILE_NAME + '.xlsx','Pages', index_col=0).reset_index()
    df2 = pd.read_excel(FILE_NAME + '.xlsx','Streets', index_col=0).reset_index()
    display(df1)

,Location ID,SEGMENT_ID,Crosswalk,Stopline,Parking 'L' parking,Parking 'T' parking,SPECIFICATIONS
0,MRK19-007170,3262005,2.0,2.0,NaN,NaN,"Install 2 crosswalk, 2 stopline"
1,MRK19-007170,3262008,2.0,2.0,NaN,NaN,"Install 2 crosswalk, 2 stopline"
2,MRK19-007170,3262009,1.0,2.0,NaN,NaN,"Install 1 crosswalk, 2 stopline"
3,MRK19-007170,3262026,NaN,1.0,NaN,NaN,Install 1 stopline
4,MRK19-007170,3262034,NaN,1.0,NaN,NaN,Install 1 stopline
5,MRK19-007170,3262040,1.0,1.0,NaN,NaN,"Install 1 crosswalk, 1 stopline"
6,MRK19-007170,3262056,2.0,2.0,NaN,NaN,"Install 2 crosswalk, 2 stopline"
7,MRK19-007170,3262096,1.0,1.0,1.0,4.0,"Install 1 crosswalk, 1 stopline, 1 parking 'l'..."
8,MRK19-007170,3262143,2.0,2.0,NaN,NaN,"Install 2 crosswalk, 2 stopline"
9,MRK19-007170,3262283,1.0,1.0,NaN,NaN,"Install 1 crosswalk, 1 stopline"


# Creating the Map

In [5]:
mxd = arcpy.mapping.MapDocument(MXD)
sde_path = r"Database Connections\GISDM.sde"

# Create Database connection 
arcpy.env.workspace = sde_path
if arcpy.Exists(sde_path) == False:
    arcpy.CreateDatabaseConnection_management("Database Connections","GISDM.sde", "ORACLE", 
                                              "sde:oracle11g:gisdm", "DATABASE_AUTH")

# Create layers
layer = arcpy.mapping.ListLayers(mxd,"TRANSPORTATION.street_segment")[0]
sl = arcpy.mapping.ListLayers(mxd,'TRANSPORTATION.markings_short_line')[0]
sp = arcpy.mapping.ListLayers(mxd,'TRANSPORTATION.markings_specialty_point')[0]

## Setting up Cover Document
A database connection to GISDM is needed to access the markings asset layers and the markings assets layers. We can open up a map document of the cover work orders and another map document of the pages work orders.

In [6]:
for index,row in df.iterrows():
    segments = str(row["Segment IDs"]).split(',')
    sql = 'SEGMENT_ID IN({})'.format(str(segments)[1:-1])
    arcpy.SelectLayerByAttribute_management(layer,"NEW_SELECTION",sql) 
    df_text(row)
    cover_aerial(mxd)
    arcpy.mapping.ExportToPDF(mxd, PLAN_PATH + "\\ATD_Whereabouts_Cover_{}.pdf".format(row["Location ID"]))
    arcpy.SelectLayerByAttribute_management(layer,"CLEAR_SELECTION")
    print "PDF file created Cover {}.pdf".format(row["Location ID"])

PDF file created Cover MRK19-007170.pdf


## Setting up Pages

In [7]:
for index,row in df1.iterrows():
    q = 'SEGMENT_ID = {}'.format(int(row["SEGMENT_ID"]))
    arcpy.SelectLayerByAttribute_management(layer,"ADD_TO_SELECTION", q)
    arcpy.SelectLayerByAttribute_management(sl, "ADD_TO_SELECTION", q)
    arcpy.SelectLayerByAttribute_management(sp, "ADD_TO_SELECTION", q)
    for e in arcpy.mapping.ListLayoutElements(mxd,'TEXT_ELEMENT'):
        if e.name == 'SPECIFICATIONS':
            e.text = str(row["SPECIFICATIONS"])
        elif e.name == 'REQUESTOR ID':
            e.text = str(row["Location ID"])
        elif e.name == 'row1':
            e.text = 'SEGMENT ID'
        elif e.name == 'LOCATION':
            e.text = str(row["SEGMENT_ID"])
        elif e.name == 'PAGE':
            b = df2['PAGE'][(df2['Location ID'] == row['Location ID']) & (df2['SEGMENT_ID'] == row['SEGMENT_ID'])]
            e.text = str(int(b))
    dataframe = arcpy.mapping.ListDataFrames(mxd,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    dataframe.scale = dataframe.scale * 1.05
    arcpy.SelectLayerByAttribute_management(sl,"CLEAR_SELECTION")
    arcpy.SelectLayerByAttribute_management(sp,"CLEAR_SELECTION")
    arcpy.SelectLayerByAttribute_management(layer,"CLEAR_SELECTION")
    arcpy.RefreshActiveView()
    arcpy.mapping.ExportToPDF(mxd, PLAN_PATH + "\\ATD_Whereabouts_Page_{}_{}.pdf".format(row["Location ID"],row['SEGMENT_ID']))
    print "PDF file created Page_{}_{}.pdf".format(row["Location ID"],row['SEGMENT_ID'])

PDF file created Page_MRK19-007170_3262005.pdf
PDF file created Page_MRK19-007170_3262008.pdf
PDF file created Page_MRK19-007170_3262009.pdf
PDF file created Page_MRK19-007170_3262026.pdf
PDF file created Page_MRK19-007170_3262034.pdf
PDF file created Page_MRK19-007170_3262040.pdf
PDF file created Page_MRK19-007170_3262056.pdf
PDF file created Page_MRK19-007170_3262096.pdf
PDF file created Page_MRK19-007170_3262143.pdf
PDF file created Page_MRK19-007170_3262283.pdf
PDF file created Page_MRK19-007170_3262287.pdf
PDF file created Page_MRK19-007170_3262305.pdf
PDF file created Page_MRK19-007170_3262309.pdf
PDF file created Page_MRK19-007170_3262808.pdf
PDF file created Page_MRK19-007170_3262813.pdf
PDF file created Page_MRK19-007170_3310370.pdf
PDF file created Page_MRK19-007170_3310390.pdf
PDF file created Page_MRK19-007170_3310405.pdf
PDF file created Page_MRK19-007170_3310423.pdf
PDF file created Page_MRK19-007170_3310428.pdf
PDF file created Page_MRK19-007170_3310435.pdf
PDF file crea

## Merging Cover and Pages

In [8]:
a = PLAN_PATH + "\\ATD_Whereabouts_"

for i,r in df.iterrows():
    merger = PdfFileMerger()
    pdf = "{}{}_{}.pdf".format(a,'Cover',r['Location ID'])
    merger.append(pdf)
    for index,row in df2.iterrows():
        if r['Location ID'] == row['Location ID']:
            pdf = "{}{}_{}_{}.pdf".format(a,'Page',row['Location ID'],row['SEGMENT_ID'])
            merger.append(pdf)
    merger.write(a + "{}_Final.pdf".format(r['Location ID']))
    merger.close()

Complete!

## Kill ArcMap Desktop

In [45]:
import os  
os.system("TASKKILL /F /IM ArcMap.exe")  

128